This will come down to a few steps:

1) Bring in the datsets from V-Dem and the Frasier Institute

2) See what entities are only included in one of the two datasets, and see what entities have different names between the two datasets

3) Make the appropriate merge with such information taken into account

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
Frasier_data = pd.read_csv("Frasier_inst_econ_data_1970.csv")
VDem_data = pd.read_csv("V-Dem_data_1970.csv")

In [3]:
print(Frasier_data.shape)
print(VDem_data.shape)

(165, 72)
(177, 7)


We can thus see that the Frasier inst data contains far fewer rows than the V-Dem data. This is likely due to V-Dem including data on countries that don't exist anymore. We can get an idea of precisely what might be missing from each as seen below:

In [4]:
Vdem_entities = VDem_data['Entity'].tolist()
Frasier_entities = Frasier_data['Countries'].tolist()

In [5]:
only_VDem=[i for i in Vdem_entities if i not in Frasier_entities]
print(only_VDem)
print(len(only_VDem))

['Afghanistan', 'Cape Verde', 'Congo', 'Cuba', 'Czechia', 'Democratic Republic of Congo', 'East Germany', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Gambia', 'Hong Kong', 'Iran', 'Kyrgyzstan', 'Laos', 'Maldives', 'North Korea', 'Palestine/West Bank', 'Republic of Vietnam', 'Russia', 'Sao Tome and Principe', 'Slovakia', 'Solomon Islands', 'South Korea', 'South Sudan', 'Syria', 'Timor', 'Turkmenistan', 'Uzbekistan', 'Vanuatu', 'Venezuela', 'Yemen', "Yemen People's Republic", 'Zanzibar']
34


Although the difference in number of countries between the two datasets is 12, V-Dem possesses 34 countries that Frasier does not possess, this would have to be due to the datasets using different names for the same countries or perhaps both possess countries that the other does not. 

In [6]:
only_Frasier=[i for i in Frasier_entities if i not in Vdem_entities]
print(only_Frasier)
print(len(only_Frasier))

['United Arab Emirates', 'Bahamas, The', 'Belize', 'Brunei Darussalam', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Cabo Verde', 'Czech Republic', 'Egypt, Arab Rep.', 'Gambia, The', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Kyrgyz Republic', 'Korea, Rep.', 'Lao PDR', 'Montenegro', 'Russian Federation', 'Slovak Republic', 'Syrian Arab Republic', 'Timor-Leste', 'Venezuela, RB', 'Yemen, Rep.']
22


In [7]:
print(sorted(only_Frasier))

['Bahamas, The', 'Belize', 'Brunei Darussalam', 'Cabo Verde', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Czech Republic', 'Egypt, Arab Rep.', 'Gambia, The', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Korea, Rep.', 'Kyrgyz Republic', 'Lao PDR', 'Montenegro', 'Russian Federation', 'Slovak Republic', 'Syrian Arab Republic', 'Timor-Leste', 'United Arab Emirates', 'Venezuela, RB', 'Yemen, Rep.']


Some of them are clearly just synonyms for the same country (e.g., Korea, Rep. vs. South Korea) For this reason a merge should not occur based on the country name but based on the ISO_Code. The biggest issue is preserving integrity in terms of what "entity" is being referred to. Regime changes occur, and countries merge, sometimes with one country being absorbed into another one, so there are issues to be aware of where it's not trivial to make sure the right entities are being matched with the right entities. ISO code clears this up by providing a formal unambiguous way of making sure what entity is what, provided that the Frasier inst and V-Dem did their due diligence. After doing the merge the entity names can be looked at to find inconsistencies and speculate if this is due to simply using different names for the same entity or a genuine error in how ISO_Codes were applied

In [8]:
#first we need to make sure both dataframes refer to the ISO_code column the same way
VDem_data.rename(columns={'Code_x_x': 'ISO_Code'}, inplace=True)
VDem_data.head()

,Unnamed: 0,Entity,ISO_Code,indiv_libs_vdem_owid,electdem_vdem_owid,libdem_vdem_owid,lib_dich_row_owid
0,0,Afghanistan,AFG,0.266,0.160,0.104,0.0
1,1,Albania,ALB,0.006,0.173,0.055,0.0
2,2,Algeria,DZA,0.428,0.084,0.065,0.0
3,3,Angola,AGO,0.024,0.010,0.035,0.0
4,4,Argentina,ARG,0.354,0.105,0.086,0.0


Furthermore Palsestine West-Bank in one of the dataframes but not the other has no ISO code so it will need to be given NA or something. A scan of the CSV's reveals that that is the only case where the ISO code is missing.

In [9]:
print(VDem_data.iloc[119, 1])
print(VDem_data.iloc[119, 2])

Palestine/West Bank
nan


It is thus cell 119, 2 that must be filled in. Given the disputed nature of the state of palestine and what territory is covered V-Dem must have left it out. I will use the following ISO-Code: PSE_? 

In [10]:
VDem_data.iloc[119, 2]='PSE_? '
print(VDem_data.iloc[119, 2])

PSE_? 


In [11]:
'''
a full outer join will be needed because even with differences in the names of countries taken into account 
there are many countries only seen in one of the two dataframes '''
merge_try = pd.merge(VDem_data,Frasier_data,on='ISO_Code',how='outer')
merge_try.head(50)

,Unnamed: 0,Entity,ISO_Code,indiv_libs_vdem_owid,electdem_vdem_owid,libdem_vdem_owid,lib_dich_row_owid,Year,Countries,Economic Freedom Summary Index,...,Conscription,Labor market regulations,Administrative requirements,Regulatory Burden,Starting a business,Impartial Public Administration,Licensing restrictions,Tax compliance,Business regulations,Regulation
0,0.0,Afghanistan,AFG,0.266,0.160,0.104,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Albania,ALB,0.006,0.173,0.055,0.0,1970.0,Albania,NaN,...,0.0,NaN,NaN,NaN,NaN,1.75,NaN,NaN,NaN,NaN
2,2.0,Algeria,DZA,0.428,0.084,0.065,0.0,1970.0,Algeria,4.68,...,3.0,NaN,NaN,NaN,NaN,3.91,NaN,NaN,NaN,3.63
3,3.0,Angola,AGO,0.024,0.010,0.035,0.0,1970.0,Angola,NaN,...,NaN,NaN,NaN,NaN,NaN,2.37,NaN,NaN,NaN,NaN
4,4.0,Argentina,ARG,0.354,0.105,0.086,0.0,1970.0,Argentina,5.98,...,0.0,3.63,NaN,NaN,NaN,2.42,NaN,NaN,NaN,5.31
5,5.0,Armenia,ARM,0.094,0.102,0.024,0.0,1970.0,Armenia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,Australia,AUS,0.980,0.860,0.807,1.0,1970.0,Australia,7.56,...,0.0,NaN,NaN,NaN,NaN,9.76,NaN,NaN,NaN,5.72
7,7.0,Austria,AUT,0.949,0.843,0.728,1.0,1970.0,Austria,6.61,...,0.0,3.17,NaN,NaN,NaN,8.06,NaN,NaN,NaN,5.16
8,8.0,Azerbaijan,AZE,0.094,0.102,0.024,0.0,1970.0,Azerbaijan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,Bahrain,BHR,0.449,0.024,NaN,NaN,1970.0,Bahrain,NaN,...,NaN,NaN,NaN,NaN,NaN,5.16,NaN,NaN,NaN,NaN


In [13]:
merge_try[merge_try['Entity'] != merge_try['Countries']]

,Unnamed: 0,Entity,ISO_Code,indiv_libs_vdem_owid,electdem_vdem_owid,libdem_vdem_owid,lib_dich_row_owid,Year,Countries,Economic Freedom Summary Index,...,Conscription,Labor market regulations,Administrative requirements,Regulatory Burden,Starting a business,Impartial Public Administration,Licensing restrictions,Tax compliance,Business regulations,Regulation
0,0.0,Afghanistan,AFG,0.266,0.160,0.104,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,26.0,Cape Verde,CPV,0.096,0.025,0.032,0.0,1970.0,Cabo Verde,NaN,...,NaN,NaN,NaN,NaN,NaN,1.76,NaN,NaN,NaN,NaN
33,33.0,Congo,COG,0.134,0.086,0.024,0.0,1970.0,"Congo, Rep.",5.04,...,NaN,NaN,NaN,NaN,NaN,1.87,NaN,NaN,NaN,NaN
37,37.0,Cuba,CUB,0.195,0.069,0.034,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,39.0,Czechia,CZE,0.230,0.132,0.046,0.0,1970.0,Czech Republic,NaN,...,0.0,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,3.07
40,40.0,Democratic Republic of Congo,COD,0.104,0.111,0.030,0.0,1970.0,"Congo, Dem. Rep.",4.06,...,3.0,NaN,NaN,NaN,NaN,2.15,NaN,NaN,NaN,3.99
44,44.0,East Germany,OWID_GDR,0.346,0.169,0.064,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,46.0,Egypt,EGY,0.430,0.193,0.148,0.0,1970.0,"Egypt, Arab Rep.",NaN,...,0.0,NaN,NaN,NaN,NaN,4.56,NaN,NaN,NaN,NaN
48,48.0,Equatorial Guinea,GNQ,0.003,0.171,0.021,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,49.0,Eritrea,ERI,0.118,0.083,0.023,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Reviewing the above view of the dataframe leads me to believe that the entities have lined up accurately. There are some cases where V-Dem and Frasier Inst use two separate names for the same country but this would seem to be due to the country having a formal name and a name more commonly used. I have no worries that different entities in reality have been falsely categorized with the same ISO code. 

In [14]:
merge_try.to_csv("V-Dem_Frasier_data_1970.csv") 